In [ ]:
import os
import tensorflow.compat.v1 as tf
import math
import numpy as np
import itertools
from scipy.spatial.transform import Rotation as R

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

import matplotlib.pyplot as plt
import open3d as o3d

In [ ]:
FILENAME = '/home/tristram/data/waymo/training-segment-10876852935525353526_1640_000_1660_000_with_camera_labels.tfrecord'
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
frames = []
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    frames.append(frame)
    

## Poses for images

In [ ]:
img_dir = '/home/tristram/data/waymo/seg3_5/images/'
for i, elem in enumerate(frames):
    if i < 5:
        for j, img in enumerate(elem.images):
            print(img.name)
            if img.name == 4:
                name = img.name - 1
            elif img.name == 3:
                name = img.name + 1
            else:
                name = img.name
            print(str(i)+'_'+str(name)+'.jpg')
            image = np.array(tf.image.decode_jpeg(img.image))
            plt.imshow(image)
            plt.axis('off')
            plt.show()
            sdir = img_dir+str(i)+'_'+str(name)+'.jpg'
            print(sdir)
            #plt.imsave(sdir, image)

In [ ]:
data_dir = '/home/tristram/data/waymo/seg2_5/'
near = 3.0
far = 200.0

opengl2waymo = np.array([[0, 0, -1, 0],
                        [-1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1]])

trafo2 = np.array([[-1, 0, 0, 0],
                        [0, 0, 1, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1]])

poses_3x5 = []
cx = []
cy = []
for i, elem in enumerate(frames):
    if i < 5:
        for j, img in enumerate(elem.images):
            if j >= 0:
                #image = tf.image.decode_jpeg(img.image).numpy()
                w = elem.context.camera_calibrations[img.name-1].width
                h = elem.context.camera_calibrations[img.name-1].height
                focal = elem.context.camera_calibrations[img.name-1].intrinsic[0]
                cx.append(elem.context.camera_calibrations[img.name-1].intrinsic[2])
                cy.append(elem.context.camera_calibrations[img.name-1].intrinsic[3])
                hwf = np.array([h, w, focal]).reshape(3,1)
                v2w = np.asarray(elem.pose.transform).reshape(4,4)
                #print(elem.context.camera_calibrations)
                c2v = np.asarray(elem.context.camera_calibrations[img.name-1].extrinsic.transform).reshape(4,4)
                #print(c2v)
                i2w = np.array(img.pose.transform).reshape(4, 4)
                img_pose = np.matmul(i2w, c2v)
                img_rot = R.from_matrix(img_pose[:3, :3])
                img_aa = img_rot.as_rotvec()
                
                #print(focal)
                pose = np.matmul(v2w, c2v)
                pose_aa = R.from_matrix(pose[:3, :3])
                pose_aa = pose_aa.as_rotvec()
                #print((np.abs(img_aa) - np.abs(pose_aa)) * (180/np.pi))
                #print(np.abs(img_pose[:3, 3]) - np.abs(pose[:3, 3]))
                #print(pose)
                pose = np.matmul(pose, opengl2waymo)
                #x = pose[0,3].copy()
                #y = pose[1,3].copy()
                #z = pose[2,3].copy()
                #pose[2,3] = y
                #pose[1,3] = z
                #pose[0,3] = -x
                pose = np.matmul(trafo2, pose)


                #pose = np.linalg.inv(pose)

                poses_3x5.append(np.hstack((pose[:3,:4], hwf)))
                #print(path+str(i)+'_'+str(j)+'.jpg')
                #plt.imsave(path+str(i)+'_'+str(j)+'.jpg', image)
        
poses_flat = np.array(poses_3x5).reshape(-1, 15)
poses_final = []
for elem in poses_flat:
    poses_final.append(np.concatenate((elem, [near, far])))
poses_final = np.array(poses_final)
pp = np.array([cx, cy]).transpose([1, 0])
poses_final = np.concatenate([poses_final, pp], axis=-1)
print(poses_final.shape)
np.save(data_dir+'poses_bounds.npy', poses_final)

## Depth Images

In [ ]:
data_dir = '/home/tristram/data/waymo/seg2_5/'
scale = 4.0

all_depth = []
for i, elem in enumerate(frames):
    if i < 5:
        for j, img in enumerate(elem.images):
            if j >= 0:
                #image = tf.image.decode_jpeg(img.image).numpy()

                (range_images,\
                 camera_projections,\
                 range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(elem)

                points, cp_points = frame_utils.convert_range_image_to_point_cloud(elem,
                                                                                   range_images,
                                                                                   camera_projections,
                                                                                   range_image_top_pose)

                # 3d points in VEHICLE frame.
                points_all = np.concatenate(points, axis=0)
                # camera projection corresponding to each point.
                cp_points_all = np.concatenate(cp_points, axis=0)
                cp_points_all[:,1] = cp_points_all[:,1] / scale
                cp_points_all[:,2] = cp_points_all[:,2] / scale
                cp_points_all[:,4] = cp_points_all[:,4] / scale
                cp_points_all[:,5] = cp_points_all[:,5] / scale

                cp_points_all_concat = np.concatenate([cp_points_all, points_all], axis=-1)
                cp_points_all_concat_tensor = tf.constant(cp_points_all_concat)

                # The distance between lidar points and vehicle frame origin.
                points_all_tensor = tf.norm(points_all, axis=-1, keepdims=True)
                cp_points_all_tensor = tf.constant(cp_points_all, dtype=tf.int32)
                                
                mask = tf.equal(cp_points_all_tensor[..., 0], img.name)
                overlap = tf.equal(cp_points_all_tensor[..., 3], img.name)

                cp_points_all_tensor_mask = tf.cast(tf.gather_nd(cp_points_all_tensor,
                                                            tf.where(mask)), dtype=tf.float32)
                cp_points_all_tensor_overlap = tf.cast(tf.gather_nd(cp_points_all_tensor,
                                                            tf.where(overlap)), dtype=tf.float32)
                
                points_all_tensor_mask = tf.gather_nd(points_all_tensor, tf.where(mask))
                points_all_tensor_overlap = tf.gather_nd(points_all_tensor, tf.where(overlap))

                projected_points_all_from_raw_data = tf.concat([cp_points_all_tensor_mask[..., 1:3],
                                                                points_all_tensor_mask], axis=-1).numpy()
                projected_point_overlap = tf.concat([cp_points_all_tensor_overlap[..., 4:6],
                                                                points_all_tensor_overlap], axis=-1).numpy()

                if projected_point_overlap.shape[0] != 0:
                    projected_points_all_from_raw_data = np.concatenate([projected_points_all_from_raw_data,
                                                                       projected_point_overlap])

                w = int(elem.context.camera_calibrations[img.name-1].width / scale)
                h = int(elem.context.camera_calibrations[img.name-1].height / scale)
                resolution = (h, w)

                depth = np.zeros(resolution)
                for pts in projected_points_all_from_raw_data:
                    if pts[0] < resolution[1] and  pts[1] < resolution[0]:
                        if pts[2] < depth[int(pts[1]), int(pts[0])] or depth[int(pts[1]), int(pts[0])] == 0:
                            depth[int(pts[1]), int(pts[0])] = pts[2]


                """img_d = visualize_depth(depth)

                plt.figure(figsize=(50,50))
                plt.imshow(img_d)
                plt.show()"""

                all_depth.append(depth)

        
all_depth = np.array(all_depth)

print(len(all_depth))
np.savez(data_dir+"depth_images.npz", all_depth)

In [ ]:
(range_images,\
camera_projections,\
range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frames[1])

points, cp_points = frame_utils.convert_range_image_to_point_cloud(frames[1],
                                                                    range_images,
                                                                    camera_projections,
                                                                    range_image_top_pose)

            # 3d points in VEHICLE frame.
points_all = np.concatenate(points, axis=0)
            # camera projection corresponding to each point.
cp_points_all = np.concatenate(cp_points, axis=0)
cp_points_all[:,1] = cp_points_all[:,1] / scale
cp_points_all[:,2] = cp_points_all[:,2] / scale
cp_points_all[:,4] = cp_points_all[:,4] / scale
cp_points_all[:,5] = cp_points_all[:,5] / scale


cp_points_all_concat = np.concatenate([cp_points_all, points_all], axis=-1)
cp_points_all_concat_tensor = tf.constant(cp_points_all_concat)

            # The distance between lidar points and vehicle frame origin.
points_all_tensor = tf.norm(points_all, axis=-1, keepdims=True)
cp_points_all_tensor = tf.constant(cp_points_all, dtype=tf.int32)

mask = tf.equal(cp_points_all_tensor[..., 0], frames[1].images[2].name)

cp_points_all_tensor = tf.cast(tf.gather_nd(cp_points_all_tensor,
                                            tf.where(mask)), dtype=tf.float32)
points_all_tensor = tf.gather_nd(points_all_tensor, tf.where(mask))

projected_points_all_from_raw_data = tf.concat([cp_points_all_tensor[..., 1:3],
                                                points_all_tensor], axis=-1).numpy()

print(projected_points_all_from_raw_data[0])

In [ ]:
depth = np.load('/home/tristram/data/waymo/seg1_2/'+'depth_images.npz', allow_pickle=True)['arr_0']


import matplotlib.pyplot as plt
import cv2
%matplotlib inline
img = cv2.imread('/home/tristram/data/waymo/seg1_2/images_4/1_2.jpg')

cmap = plt.cm.get_cmap("hsv", 256)
cmap = np.array([cmap(i) for i in range(256)])[:, :3] * 255

for i in range(projected_points_all_from_raw_data.shape[0]):
    depth = projected_points_all_from_raw_data[i, 2]
    color = cmap[int(640.0 / depth), :]
    cv2.circle(
        img,
        (int(np.rint(projected_points_all_from_raw_data[i, 0])), int(np.rint(projected_points_all_from_raw_data[i, 1]))),
        0,
        color=tuple(color),
        thickness=-1,
        )

plt.figure(figsize=(50,50))
plt.imshow(img)
plt.show()

In [ ]:
h = np.linspace(0, 320, num=320, dtype=np.int)
w = np.linspace(0, 480, num=480, dtype=np.int)

img = np.meshgrid(w, h)

depth = np.load('/home/tristram/data/waymo/seg1_5/'+'depth_images.npz', allow_pickle=True)['arr_0']

pts = np.zeros((320, 480, 3))
pts[:,:,0] = img[0]
pts[:,:,1] = img[1]
pts[:,:,2] = depth[1]

pts = pts.reshape(-1,3)
pts[pts==0.0] = -1000.0

test = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))
o3d.io.write_point_cloud('/home/tristram/nerf_results/WAYMO1_5_ds_3_200_log_4x128/' + "test0.ply", test)

#plt.imshow(depth[1])
#plt.show

In [ ]:
data_dir = '/home/tristram/data/waymo/seg1_5/'

sky = np.load('/home/tristram/data/waymo/seg1_20/'+'sky_masks.npz', allow_pickle=True)['arr_0']
save = []
for i, elem in enumerate(sky):
    if i < 25:
        save.append(elem)

save = np.array(save)
print(save.shape)
np.savez(data_dir+'sky_masks.npz', save)

In [ ]:
load = np.load('/home/tristram/data/waymo/seg2_5/'+'depth_images.npz', allow_pickle=True)['arr_0']

for l in load:
    
    img = visualize_depth(l)
    print(l.shape)
    plt.figure(figsize=(50,50))
    plt.imshow(img)
    plt.show()

In [ ]:


imgdir = data_dir + 'images_4/'
if not utils.file_exists(imgdir):
    raise ValueError('Image folder {} does not exist.'.format(imgdir))
imgfiles = [
    path.join(imgdir, f) for f in natsorted(utils.listdir(imgdir))
    if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')
        ]

images = []
for imgfile in imgfiles:
    with utils.open_file(imgfile, 'rb') as imgin:
        image = np.array(Image.open(imgin), dtype=np.float32) / 255.
        images.append(image)
images = np.array(images, dtype=object)

plt.figure()
plt.imshow(images[4])
plt.show()

plt.figure()
plt.imshow(images[9])
plt.show()

In [ ]:
veh = np.load(data_dir + 'poses_bounds.npy')
cams = veh[:, :15].reshape(-1,3,5).copy()
cc = cams[:,:3,:4].copy()
cams_centered, _ = center_poses(cc)
cams[:,:3,:4] = cams_centered
cams = cams.reshape(-1,15)
veh[:,:15] = cams

print(cams_centered[0])
print(poses1[0])
print(poses2[0, :3, :4])

#np.save(data_dir+'test_poses.npy', veh)

In [ ]:
depth = np.load('/home/tristram/data/waymo/seg1_10_0/'+'depth_images.npy')
print(len(depth))
print(depth.shape)
print(depth[0])
img_d = visualize_depth(depth[0])
plt.figure(figsize=(50,50))
plt.imshow(img_d)
plt.show()

img_d = visualize_depth(depth[1])
plt.figure(figsize=(50,50))
plt.imshow(img_d)
plt.show()

## Utilities

In [ ]:
# CHECK IF SHAPES MATCH

from PIL import Image
from natsort import natsorted
path = '/home/tristram/data/waymo/seg1_20/images_4/'

imgs = natsorted(os.listdir(path))
poses = np.load('/home/tristram/data/waymo/seg1_20/poses_bounds.npy')[:,:15]
poses = poses.reshape(-1, 3, 5)
depth = np.load('/home/tristram/data/waymo/seg1_20/depth_images.npz', allow_pickle=True)['arr_0']
sky = np.load('/home/tristram/data/waymo/seg1_20/sky_masks.npz', allow_pickle=True)['arr_0']

for i, name in enumerate(imgs):
    img = np.array(Image.open(path+name))
    if img.shape[0] != int(poses[i, 0, -1] / 4):
        print(name)
    if img.shape[1] != int(poses[i, 1, -1] / 4):
        print(name)
    if img.shape[:2] != depth[i].shape:
        print(name)
    if img.shape[:2] != sky[i].shape:
        print(name)
    

In [ ]:
import numpy as np
import cv2
from PIL import Image

def visualize_depth(depth, cmap=cv2.COLORMAP_TWILIGHT_SHIFTED):
    """
    depth: (H, W)
    """
    x = depth
    
    x = np.nan_to_num(x) # change nan to 0
    mi = np.min(x) # get minimum depth
    ma = np.max(x)
    x = (x-mi)/(ma-mi+1e-8) # normalize to 0~1
    x = (255*x).astype(np.uint8)
    x_ = Image.fromarray(cv2.applyColorMap(x, cmap))
    return x_

In [ ]:
import shutil
from natsort import natsorted

old = '/home/tristram/data/waymo/segment1/images_4/'
names = natsorted(os.listdir('/home/tristram/data/waymo/segment1/images/'))
new = '/home/tristram/data/waymo/seg1_5/images_4/'

for i, elem in enumerate(names):
    if i < 5 * 5:
        shutil.copy(old+elem, new+elem)

In [ ]:
import open3d as o3d

(range_images,\
camera_projections,\
range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frames[0])



# points are given in vehicle coordinates
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frames[0],
    range_images,
    camera_projections,
    range_image_top_pose)
points_all = np.concatenate(points, axis=0)
points_hom = np.ones((points_all.shape[0], 4))
points_hom[:,:3] = points_all

# this transforms from vehicle to world coordinates
v2w = np.asarray(frames[0].pose.transform).reshape(4,4)

opengl2waymo = np.array([[0, 0, -1, 0],
                        [-1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1]])

# transform points to world coordinates
points_w = np.matmul(points_hom, v2w.T)
points_wgl = np.matmul(points_w, opengl2waymo)

data_dir = '/home/tristram/data/waymo/segment1/'

test = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points_wgl[:,:3]))
o3d.io.write_point_cloud(data_dir + "test0.ply", test)